# Ejercicio de curvas elípticas

Sea $\mathbb{F}_32 = \mathbb{F}_2[\xi]_{\xi^5 + \xi^2 + 1}$. Cada uno de vosotros, de acuerdo a su número de DNI, dispone de una curva elíptica sobre $\mathbb{F}_{32}$ y un punto base dados en el cuadro 6.1.

1. Calcula mediante el algoritmo de Shank o el 9, $log_Q{\mathcal{O}}$.
2. Para tu curva y tu punto base, genera un par de claves pública/privada para un protocolo ECDH.
3. Cifra el mensaje $(\xi^3 + \xi^2 + 1, \xi^4 + \xi^2) \in \mathbb{F}^2_{32}$.
4. Descifra el mensaje anterior.

Como mi DNI es

In [1]:
dni = 77432071

Tenemos que $\text{dni} \bmod 32$ da 

In [2]:
dni.mod(32)

7

Por lo que mis curvas son:

$$
\begin{aligned}
E(A, B) & = E(\xi + 1, \xi^4 + \xi^3 + \xi + 1) \\ 
Q & = (\xi^3 + \xi^2 + \xi, \xi^4 + \xi^3 + \xi^2 + \xi + 1)
\end{aligned}
$$

In [15]:
F.<xi> = GF(2^5, modulus = GF(2)[x](x^5 + x^2 + 1))
E = EllipticCurve(F, [1, xi + 1, 0, 0, xi^4 + xi^3 + xi + 1])
Q = E.point((xi^3 + xi^2 + xi, xi^4 + xi^3 + xi^2 + xi + 1))

In [16]:
E

Elliptic Curve defined by y^2 + x*y = x^3 + (xi+1)*x^2 + (xi^4+xi^3+xi+1) over Finite Field in xi of size 2^5

In [17]:
Q

(xi^3 + xi^2 + xi : xi^4 + xi^3 + xi^2 + xi + 1 : 1)

## Apartado 1

## Apartado 2

Para el algoritmo ECDH, fijamos $E = E(a, b)$ de forma que $\|E\| = hn$, con $n$ primo, $h$ pequeño y $\|Q\| = n$.

In [19]:
E.order()

34

In [20]:
Q.order()

17

Podemos tomar 

In [41]:
n = 17
h = 2

In [42]:
n * h == E.order() and n == Q.order()

True

Ahora Alice debe escoger un número aleatorio $a$ que se encuentre entre $2 \le a \le n-1$, y hacer público $(E, Q, aQ)$

In [43]:
a = ZZ.random_element(2, n)

In [44]:
clave_alice = a*Q
clave_alice

(xi^3 + xi : xi^4 + xi^3 + xi^2 + xi + 1 : 1)

Vamos ahora a generar la clave privada. Para ello, Bob escoge $k$ tal que $2 \le k \le n-1$

In [45]:
k = ZZ.random_element(2, n)

Por lo que 

In [50]:
clave_bob = k * Q
clave_bob

(xi^4 + xi^3 + xi^2 + xi : xi^2 + xi : 1)

Bob transmitirá la siguiente clave:

In [54]:
clave_bob_compartida = a * clave_bob
clave_bob_compartida

(xi^3 + xi^2 + xi : xi^4 + 1 : 1)

Así que, en conjunto, las claves son

In [56]:
clave_alice, clave_bob, clave_bob_compartida

((xi^3 + xi : xi^4 + xi^3 + xi^2 + xi + 1 : 1),
 (xi^4 + xi^3 + xi^2 + xi : xi^2 + xi : 1),
 (xi^3 + xi^2 + xi : xi^4 + 1 : 1))

Podemos verificar que se han escogido correctamente: 

In [48]:
a * clave_bob == k * clave_alice

True

## Apartado 3

El mensaje $M = (m_1, m_2)$ es

In [59]:
mensaje = [xi^3 + xi^2 + 1, xi^4 + xi^2]
mensaje

[xi^3 + xi^2 + 1, xi^4 + xi^2]

Podemos partir la clave en dos de la forma $(x_0, y_0) = k(aQ)$:

In [112]:
(k*(a*Q))[0], (k*(a*Q))[1]

(xi^3 + xi^2 + xi, xi^4 + 1)

El mensaje que se va a transmitir es 

$$
\begin{aligned}
E(M) & = E(m_1, m_2) = (kQ, M + k(aQ)) = \\ & = (kQ, x_0m_1, y_0m_2)
\end{aligned}
$$

In [113]:
x0m1 = mensaje[0] + (k*(a*Q))[0]
y0m2 = mensaje[1] + (k*(a*Q))[1]

print(
        "kQ      = " + str(k*Q) 
    + "\nx_0 m_1 = " + str(x0m1) 
    + "\ny_0 m_2 = " + str(y0m2)
)

kQ      = (xi^4 + xi^3 + xi^2 + xi : xi^2 + xi : 1)
x_0 m_1 = xi + 1
y_0 m_2 = xi^2 + 1


## Apartado 4

Para descifrar el mensaje, Alice debe hacer

$$
\begin{aligned}
D(kQ, M + k(aQ)) & = M + k(aQ) - a(kQ) = \\ 
                 & = M
\end{aligned}
$$

In [121]:
descifrado = [
    x0m1 - (a*(k*Q))[0], 
    y0m2 - (a*(k*Q))[1]
]
descifrado

[xi^3 + xi^2 + 1, xi^4 + xi^2]

In [122]:
descifrado == mensaje

True